In [2]:
from flask import Flask, request, jsonify
import threading
import cv2
from ultralytics import YOLO
import numpy as np
import joblib
from datetime import datetime
import os
import requests
import json
app = Flask(__name__)
video_capture = None
capture_thread = None
running = False

def start_capture():
    global running, video_capture
    video_capture = cv2.VideoCapture(0)
    running = True
    while running:
        ret, frame = video_capture.read()
        if not ret:
            break
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
    video_capture.release()
    cv2.destroyAllWindows()

@app.route('/start', methods=['GET'])
def start():
    global capture_thread
    if not running:
        capture_thread = threading.Thread(target=start_capture)
        capture_thread.start()
        return jsonify({"status": "Capture started"}), 200
    else:
        return jsonify({"status": "Capture is already running"}), 400

@app.route('/stop', methods=['GET'])
def stop():
    global running
    if running:
        running = False
        capture_thread.join()
        return jsonify({"status": "Capture stopped"}), 200
    else:
        return jsonify({"status": "Capture is not running"}), 400

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\User\anaconda3\envs\DL\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import joblib
from datetime import datetime
import os
import requests
import json

# 初始化模型和其他資源
model = YOLO("yolov8n-pose.pt")
knn_model = joblib.load('knn_model.pkl')
knn_scaler = joblib.load('knn_scaler.pkl')
path = "./fall.mp4"
cap = cv2.VideoCapture(path)

# LINE Notify token
token = '5RYO1QtkRHIJ5UH9GmC8heZYJ30XBsnqkChyIg3lTO0'

# 伺服器端點的URL
server_url = 'http://192.168.24.94:8081/api/pictures'

# 將信心值加入關鍵點座標
def process_image(results):
    r = results[0]
    combined_results = []
    for i in range(len(r.boxes)):
        box_results = []
        for j in range(17):
            x, y = r.keypoints.xyn[i][j].numpy()
            box_results.extend([x, y])
        combined_results.append(box_results)
    return combined_results

# LINE Notify傳送訊息
def send_line_notify(token, image_path=None):
    message = '有人跌倒了'
    headers = {"Authorization": "Bearer " + token}
    data = {'message': message}
    if image_path:
        with open(image_path, 'rb') as image_file:
            image_data = {'imageFile': image_file}
            response = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data, files=image_data)
    else:
        response = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data)
    print(response.status_code)
    print(response.text)

# 發送跌倒事件到伺服器，並上傳圖片
def send_fall_event_to_server(timestamp, kp, image_path=None):
    # 準備 JSON 數據
    data = {
        'createdAt': timestamp,
        'points': kp.tolist() 
    }
    
    # 如果有圖片路徑，則讀取圖片並準備上傳
    files = None
    if image_path:
        files = {'image': open(image_path, 'rb')}
    
    # 發送 POST 請求
    headers = {'Content-Type': 'application/json'}
    response = requests.post(server_url, 
                             data=json.dumps(data), 
                             headers=headers, 
                             files=files)
    
    # 處理伺服器的回應
    if response.status_code == 200:
        print(f'事件成功發送: {timestamp}')
    else:
        print(f'事件發送失敗: {response.status_code}, {response.text}')


# 開始影片偵測
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        current_time = datetime.now()
        # Perform pose detection
        results = model.track(frame, conf=0.7)
        annotated_frame = results[0].plot()
        r = results[0]
        if len(results[0].boxes) >= 1:
            data = np.array(process_image(results))
            data = knn_scaler.transform(data)
            y_pred = knn_model.predict(data)
            print(y_pred)
            kp = data.reshape(-1, 2)
            print(kp)
            if y_pred[0] == 2:
                fall_current_time = current_time
                folder_name = 'Fall_img'
                os.makedirs(folder_name, exist_ok=True)
                time_text = fall_current_time.strftime("%Y-%m-%dT%H:%M:%S")
                image_filename = fall_current_time.strftime("%Y-%m-%d_%H%M%S_fall.jpg")
                cv2.imwrite(os.path.join(folder_name, image_filename), frame)
                print("output save")
                # 通知LINE，並將圖片送出
                #image_path = os.path.join(folder_name, image_filename)
                #send_line_notify(token, image_path=image_path)
                # 發送跌倒事件到伺服器
                send_fall_event_to_server(time_text, kp)
        cv2.imshow('YOLOv8 Pose Detection', annotated_frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
except KeyboardInterrupt:
    print("Script terminated by user")

cap.release()
cv2.destroyAllWindows()
